<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a2a530960edc584887837e0e2772275b8bd8aa360c5a575d077aaf2db14fea46
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-14 09:17:07
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.41 C
-------------------
Today PnL: 29.13 K (0.21%)
Current PnL: -20.84 L (-13.88%)
CY Booked + Current PnL: -8.94 L (-5.96%)
-------------------
Total profit:  1.42 L
Total loss:  -22.26 L
-------------------
Total Booked + Current PnL: 18.04 L (14.65%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.46%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.04 L (62.6%)
Deployed:  1.23 C
Current:  1.41 C
CAGR/XIRR %: 7.94%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-20.12,56.0,X-LC,3.74,213254.0,8754.0,14544.0,-0.45,4.28,6.82,11.39,10.0,0.60,1.53,13.62,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,94.02,42.0,M-SC,1.36,83630.0,-17147.0,17236.0,0.16,-17.01,20.61,0.09,245.0,-0.99,0.60,9.87,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,-0.52,53.0,X-MC,1.03,208620.0,16878.0,20883.0,0.89,8.80,10.01,19.69,99.0,0.81,1.49,16.20,XY25,NTT,AC
56,RELIANCE,1533.00,-13.66,51.0,X-LC,2.86,215779.0,5413.0,23369.0,0.60,2.57,10.83,13.68,37.0,0.23,1.55,19.52,XY25,NTT,REFINERIES
41,ITC,452.00,-39.19,43.0,X-LC,2.71,195796.0,-4342.0,25238.0,0.29,-2.17,12.89,10.44,4.0,-0.17,1.40,3.27,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,38.75,71.0,M-LC,8.72,225982.0,20906.0,91658.0,2.76,10.19,40.56,54.89,52.0,0.23,1.62,36.75,X5K,ATH,METALS
29,HCLTECH,1937.36,4.52,68.0,X-LC,9.03,238656.0,-3268.0,65511.0,1.70,-1.35,27.45,25.73,8.0,-0.05,1.71,12.78,X40,ATH,IT
4,ANGELONE,3033.00,9.69,62.0,X-SC,10.35,194455.0,3449.0,54253.0,1.31,1.81,27.90,30.21,157.0,0.06,1.39,23.06,X40N,NTT,FINANCE
17,CERA,9475.00,-23.15,43.0,H-SC,1.97,140024.0,-35879.0,77895.0,1.26,-20.40,55.63,23.89,149.0,-0.46,1.00,20.93,OX40N,NTT,CERAMICS
59,SAIL,228.00,46.75,54.0,M-MC,12.70,230989.0,6027.0,161623.0,1.23,2.68,69.97,74.53,192.0,0.04,1.66,35.80,XY24,BTT,STEEL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.0,-24.14,37.0,X-LC,1.17,239852.0,-36671.0,146741.0,-2.68,-13.26,61.18,39.81,54.0,-0.25,1.72,14.95,XY24,NTT,AUTO
30,HINDUNILVR,2922.0,-11.16,39.0,X-LC,7.89,249280.0,-7783.0,42926.0,-1.43,-3.03,17.22,13.67,24.0,-0.18,1.79,16.76,XY25,NTT,FMCG
23,DEN,75.0,94.30,42.0,M-SC,3.75,54400.0,-30064.0,65601.0,-1.22,-35.59,120.59,42.07,237.0,-0.46,0.39,15.53,AR,NTT,ENTERTAINMENT
7,ATULAUTO,844.0,3608.11,49.0,M-SC,4.73,166515.0,-30617.0,120440.0,-0.59,-15.53,72.33,45.57,236.0,-0.25,1.19,18.05,XY24,NTT,AUTO
15,CAMPUS,393.0,-22.43,62.0,M-SC,2.62,161482.0,-6293.0,62138.0,-0.56,-3.75,38.48,33.28,210.0,-0.10,1.16,25.16,XY24,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,108.87,64.0,H-SC,11.67,131895.0,-10584.0,30098.0,0.23,-7.43,22.82,13.70,163.0,-0.35,0.95,55.67,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,94.02,42.0,M-SC,1.36,83630.0,-17147.0,17236.0,0.16,-17.01,20.61,0.09,245.0,-0.99,0.60,9.87,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,48.0,H-SC,2.06,220662.0,-49005.0,85330.0,-0.21,-18.17,38.67,13.47,138.0,-0.57,1.58,12.20,XY24,NTT,PAINTS
17,CERA,9475.0,-23.15,43.0,H-SC,1.97,140024.0,-35879.0,77895.0,1.26,-20.40,55.63,23.89,149.0,-0.46,1.00,20.93,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.38,45.0,H-MC,6.98,102306.0,-29529.0,71757.0,0.61,-22.40,70.14,32.03,98.0,-0.41,0.73,14.59,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,46.75,54.0,M-MC,12.70,230989.0,6027.0,161623.0,1.23,2.68,69.97,74.53,192.0,0.04,1.66,35.80,XY24,BTT,STEEL
54,RECLTD,446.00,45.36,47.0,M-LC,6.18,205260.0,2090.0,40046.0,0.57,1.03,19.51,20.74,55.0,0.05,1.47,6.63,XY25,NTT,FINANCE
31,HINDZINC,730.22,38.75,71.0,M-LC,8.72,225982.0,20906.0,91658.0,2.76,10.19,40.56,54.89,52.0,0.23,1.62,36.75,X5K,ATH,METALS
78,UNIONBANK,163.00,-11.63,59.0,M-LC,8.75,159169.0,18329.0,26167.0,0.06,13.01,16.44,31.59,66.0,0.70,1.14,42.95,XY24,NTT,BANKS
85,WIPRO,420.00,-12.89,52.0,M-LC,5.97,153035.0,2090.0,107369.0,0.69,1.38,70.16,72.51,53.0,0.02,1.10,7.49,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,38.75,71.0,M-LC,8.72,225982.0,20906.0,91658.0,2.76,10.19,40.56,54.89,52.0,0.23,1.62,36.75,X5K,ATH,METALS
11,BANKINDIA,190.00,-24.22,69.0,H-MC,12.78,195203.0,15395.0,97016.0,0.05,8.56,49.70,62.52,88.0,0.16,1.40,45.57,XR,NTT,BANKS
36,INDIAMART,4810.62,-54.64,42.0,H-SC,8.34,126596.0,3260.0,128368.0,-0.41,2.64,101.40,106.72,119.0,0.03,0.91,26.64,AR,ATH,MISC
85,WIPRO,420.00,-12.89,52.0,M-LC,5.97,153035.0,2090.0,107369.0,0.69,1.38,70.16,72.51,53.0,0.02,1.10,7.49,XR,NTT,IT
84,WHIRLPOOL,2270.00,-48.24,32.0,M-SC,3.38,87562.0,-3936.0,82685.0,0.28,-4.30,94.43,86.07,223.0,-0.05,0.63,27.16,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,32.0,X-SC,37.80,52232.0,-12774.0,163706.0,0.00,-19.65,313.42,232.18,198.0,-0.08,0.37,0.21,XY24,NTT,MISC
28,HAVELLS,2069.16,-7.03,35.0,X-MC,1.28,230381.0,-17350.0,92406.0,0.25,-7.00,40.11,30.30,92.0,-0.19,1.65,5.36,X40,ATH,ELECTRICAL
22,DABUR,735.00,-12.48,36.0,X-MC,3.40,236995.0,-11073.0,119469.0,0.18,-4.46,50.41,43.70,102.0,-0.09,1.70,9.34,XY24,BTT,FMCG
5,ASIANPAINT,3465.66,-18.16,36.0,X-LC,5.34,206703.0,-45065.0,98267.0,0.27,-17.90,47.54,21.13,27.0,-0.46,1.48,11.18,X40,ATH,PAINTS
9,BAJAJHFL,181.50,-15.11,36.0,X-MC,5.44,178957.0,-21549.0,118702.0,0.37,-10.75,66.33,48.45,90.0,-0.18,1.28,3.20,X40N,ATH,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.68,42.0,H-LC,0.93,154940.0,-31155.0,78632.0,0.69,-16.74,50.75,25.51,15.0,-0.40,1.11,14.18,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,-0.52,53.0,X-MC,1.03,208620.0,16878.0,20883.0,0.89,8.80,10.01,19.69,99.0,0.81,1.49,16.20,XY25,NTT,AC
72,TATAMOTORS,1065.00,-24.14,37.0,X-LC,1.17,239852.0,-36671.0,146741.0,-2.68,-13.26,61.18,39.81,54.0,-0.25,1.72,14.95,XY24,NTT,AUTO
28,HAVELLS,2069.16,-7.03,35.0,X-MC,1.28,230381.0,-17350.0,92406.0,0.25,-7.00,40.11,30.30,92.0,-0.19,1.65,5.36,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-20.81,39.0,X-MC,1.46,136138.0,5.0,31121.0,0.34,0.00,22.86,22.86,91.0,0.00,0.98,12.62,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,32.0,X-SC,37.80,52232.0,-12774.0,163706.0,0.00,-19.65,313.42,232.18,198.0,-0.08,0.37,0.21,XY24,NTT,MISC
82,VBL,671.64,-21.40,39.0,X-LC,10.48,283272.0,-32570.0,145234.0,0.35,-10.31,51.27,35.67,5.0,-0.22,2.03,2.11,X40N,ATH,FMCG
21,COLPAL,3767.14,-4.98,38.0,X-MC,7.64,222020.0,-41345.0,154704.0,-0.05,-15.70,69.68,43.04,84.0,-0.27,1.59,3.07,XY25,ATH,FMCG
9,BAJAJHFL,181.50,-15.11,36.0,X-MC,5.44,178957.0,-21549.0,118702.0,0.37,-10.75,66.33,48.45,90.0,-0.18,1.28,3.20,X40N,ATH,FINANCE
41,ITC,452.00,-39.19,43.0,X-LC,2.71,195796.0,-4342.0,25238.0,0.29,-2.17,12.89,10.44,4.0,-0.17,1.40,3.27,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,32.0,X-SC,37.80,52232.0,-12774.0,163706.0,0.00,-19.65,313.42,232.18,198.0,-0.08,0.37,0.21,XY24,NTT,MISC
55,RELAXO,1176.00,-40.27,38.0,X-SC,3.26,81451.0,-63709.0,139640.0,0.30,-43.89,171.44,52.31,136.0,-0.46,0.58,9.03,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-35.87,40.0,X-SC,4.31,95832.0,-32838.0,78132.0,0.49,-25.52,81.53,35.20,219.0,-0.42,0.69,9.99,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-4.72,52.0,X-SC,1.20,108312.0,6308.0,37617.0,0.19,6.18,34.73,43.06,122.0,0.17,0.78,24.36,X40N,ATH,MISC
32,HONAUT,58357.33,-23.00,45.0,X-SC,2.22,108480.0,-19458.0,66596.0,0.01,-15.21,61.39,36.84,143.0,-0.29,0.78,11.06,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4406.34,-29.01,49.0,X-LC,12.25,283363.0,-62593.0,130829.0,0.24,-18.09,46.17,19.73,1.0,-0.48,2.03,4.37,X40,ATH,IT
39,INFY,2275.00,-17.80,54.0,X-LC,7.49,320458.0,7570.0,164107.0,0.76,2.42,51.21,54.87,3.0,0.05,2.30,9.23,X40,BTT,IT
41,ITC,452.00,-39.19,43.0,X-LC,2.71,195796.0,-4342.0,25238.0,0.29,-2.17,12.89,10.44,4.0,-0.17,1.40,3.27,X40,NTT,FMCG
82,VBL,671.64,-21.40,39.0,X-LC,10.48,283272.0,-32570.0,145234.0,0.35,-10.31,51.27,35.67,5.0,-0.22,2.03,2.11,X40N,ATH,FMCG
1,ABB,7934.00,-40.84,43.0,H-LC,7.27,247080.0,-14539.0,133744.0,0.00,-5.56,54.13,45.57,7.0,-0.11,1.77,5.13,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,80.00,-45.67,50.0,L-SC,28.74,81773.0,-31776.0,71829.0,0.00,-27.98,87.84,35.27,268.0,-0.44,0.59,100.14,XR,NTT,HOTELS
48,MASFIN,398.61,-20.51,42.0,H-SC,9.93,90405.0,-7575.0,29174.0,-0.17,-7.73,32.27,22.05,152.0,-0.26,0.65,31.41,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,108.87,64.0,H-SC,11.67,131895.0,-10584.0,30098.0,0.23,-7.43,22.82,13.70,163.0,-0.35,0.95,55.67,OX40N,NTT,BANKS
25,GREENPANEL,537.00,224.94,52.0,M-SC,4.60,144960.0,-38118.0,117099.0,0.63,-20.82,80.78,43.14,230.0,-0.33,1.04,36.36,XY24,NTT,MISC
60,SAMMAANCAP,326.00,-184.09,65.0,M-SC,17.41,146115.0,-3105.0,147284.0,-0.04,-2.08,100.80,96.62,208.0,-0.02,1.05,58.13,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-184.09,65.0,M-SC,17.41,146115.0,-3105.0,147284.0,-0.04,-2.08,100.80,96.62,208.0,-0.02,1.05,58.13,XY25,NTT,FINANCE
68,SURYODAY,240.00,64.47,61.0,H-SC,10.68,151062.0,-28009.0,93734.0,0.49,-15.64,62.05,36.71,135.0,-0.30,1.08,49.88,XR,NTT,BANKS
31,HINDZINC,730.22,38.75,71.0,M-LC,8.72,225982.0,20906.0,91658.0,2.76,10.19,40.56,54.89,52.0,0.23,1.62,36.75,X5K,ATH,METALS
63,SHALBY,327.00,1195.06,58.0,M-SC,23.04,176057.0,-5086.0,50546.0,-0.31,-2.81,28.71,25.10,235.0,-0.10,1.26,41.09,XY24,NTT,HEALTHCARE
67,STARHEALTH,761.00,16.61,66.0,H-SC,8.17,259869.0,-21254.0,154128.0,-0.42,-7.56,59.31,47.26,144.0,-0.14,1.86,39.23,XY24,NTT,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.29
1,25,43.89
2,50,75.96


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.29
LC    34.08
MC    21.66
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.22
X40      14.73
XY25     11.95
XR       10.98
X40N     10.00
AR        8.18
OX40N     7.64
X200      1.78
X5K       1.62
SR        1.12
MH        0.81
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.63
X-LC    23.07
X-MC    15.90
M-SC    12.63
M-LC     5.33
H-LC     4.66
X-SC     4.59
H-MC     3.79
M-MC     1.66
L-SC     1.44
L-LC     1.02
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.85,-6.19,41.20
IT,12.98,-16.19,79.11
FINANCE,12.07,-12.30,63.20
MISC,6.96,-18.52,84.19
BANKS,6.49,-10.20,68.17
PAINTS,5.53,-19.57,37.39
ELECTRICAL,5.31,-11.14,50.13
INSURANCE,3.81,-4.12,43.04
AUTO,3.46,-19.13,76.08


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3216491.0,22
XR,1378949.0,14
AR,1190210.0,9
X40,785703.0,10
X40N,632523.0,8
OX40N,579916.0,10
XY25,515849.0,8
SR,265004.0,2
X5K,91658.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3424921.0,25
M-SC,1281010.0,15
X-MC,1107321.0,11
X-LC,1022789.0,13
X-SC,539944.0,6
H-MC,350279.0,3
H-LC,289480.0,3
M-LC,265240.0,4
L-SC,260209.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1194598.0      6
           XR         812874.0      7
           AR         812338.0      5
M-SC       XY24       686855.0      6
X-MC       XY24       589413.0      4
X-LC       X40        483952.0      5
X-SC       X40N       298106.0      4
H-SC       OX40N      269027.0      4
           SR         265004.0      2
X-MC       X40        223619.0      4
X-LC       X40N       215715.0      3
           XY24       212623.0      2
H-LC       AR         212376.0      2
H-MC       XY24       181506.0      1
X-MC       XY25       175587.0      2
L-SC       XR         171059.0      2
M-SC       AR         165496.0      2
X-SC       XY24       163706.0      1
M-MC       XY24       161623.0      1
M-SC       OX40N      149982.0      4
           XY25       147284.0      1
           XR         131393.0      2
X-MC       X40N       118702.0      1
X-LC       XY25       110499.0      3
M-LC       XR         107369.0      1
H-MC       XR          97016.0      1
M-LC       X5K         91658.0      1
L-SC       OX40N       89150.0      1
X-SC       X40         78132.0      1
H-LC       X200        77104.0      1
H-MC       OX40N       71757.0      1
H-SC       MH          71080.0      1
L-MC       XR          59238.0      1
L-LC       XY25        42433.0      1
M-LC       XY25        40046.0      1
           XY24        26167.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
